# Auswertung: Vegetativer Bedeckungsgrad (FVC)

Nachdem die Nadir-Bilder in zwei Korrekturschritten aufgearbeitet wurden, erfolgt nun im dritten Notebook die Auswertung. Da die Nadir-Bilder keine multispektralen Daten sind, kann die Vegetation in ihnen nicht durch Vegetationsindizes (z.B. NDVI) vom Untergrund separiert werden. Die einzige Möglichkeit die Vegetation zu detektieren ist die Farbdetektion.

Dafür werden die Bilder vom RGB-Farbraum in den HSV-Farbraum konvertiert und anschließend mithilfe einer colour ramp maskiert. Der FVC wird dann wie folgt berechnet: Anzahl maskierte Pixel / (Gesamtanzahl Pixel des Bildes x 0,9).

Die Multiplikation der Gesamtanzahl der Pixel mit 0,9 dient der Berücksichtigung der Bonitätsrahmensbestandteile (~10%).

<b>Aktuell ist nur die Detektion von grüner Vegetation möglich.</b>

In [ ]:
import cv2, glob, os, datetime
import numpy as np
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display

img_name_list = []
fvc_list = []

output2_path = r"./output2/" # Pfad der sekundär (Rahmenfehlstellung) korrigierten Nadir-Bildern
vegetation_path = r"./output3/" # Pfad der ausgewerteten Nadir-Bilder mit maskierter Vegetation
no_vegetation_path = r"./output3/no vegetation/" # Pfad der ausgegebenen Nadir-Bilder mit maskierter nicht-Vegetation

# Definition der colour ramp
#light_green = (40, 40, 40)
light_green = (30, 20, 20) # hellste möglicher HSV-Grünton
dark_green = (100, 255, 255) # dunkelster möglicher HSV-Grünton

nadir_images = glob.glob(output2_path + "*.jpg")

f = IntProgress(min=0, max=len(nadir_images), description='Auswertung:',bar_style='success')
display(f)

# Sämtliche JPGs der sekundär korrigierten Nadir-Bilder werden eingelesen
for nadir_img in nadir_images:
    print("Auswertung: " + nadir_img)
    img = cv2.imread(nadir_img)
    img_name_list.append(nadir_img[10:])
    
    # Umwandlung von BGR(OpenCV) zu RGB
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Umwandlung von RGB zu HSV
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)
    
    # Ertzeugen der colour ramp
    mask = cv2.inRange(hsv_img, light_green, dark_green)
    
    # Maskieren und Anzahl detektierter Vegetationspixel
    vegetation_pixels = cv2.countNonZero(mask)

    height = img.shape[0] # Höhe
    width = img.shape[1] # Breite

    # Gesamtanzahl der Pixel des Bilds
    pixels_in_img = height * width

    # Berechnung FVC mit Berücksichtigung Rahmen (~10%)
    fvc = (vegetation_pixels/(pixels_in_img * 0.9))*100
    fvc_list.append(round(fvc, 2))
    
    # Maskieren für den Output
    vegetation = cv2.bitwise_and(img, img, mask=mask)
    no_vegetation = cv2.bitwise_and(img, img, mask = 255 - mask)

    if not os.path.isdir(vegetation_path):
        os.mkdir(vegetation_path)
        
    if not os.path.isdir(no_vegetation_path):
        os.mkdir(no_vegetation_path)
        
    cv2.imwrite(vegetation_path + nadir_img[10:], vegetation)
    cv2.imwrite(no_vegetation_path + nadir_img[10:], no_vegetation)
    
    f.value += 1

datum = datetime.datetime.now()
datum_str = str(datum.day) + "-" + str(datum.month) + "-" + str(datum.year) + "-" + str(datum.hour) + "-" + str(datum.minute) + "-" + str(datum.second)
excel_file = "FVC_" + datum_str + ".xlsx"

# Ausgabe als Excel-Datei
fvc_dict = {"Nadir-Bild":img_name_list, "FVC [%]":fvc_list}
fvc_df = pd.DataFrame(fvc_dict)
fvc_df = fvc_df.sort_values(by=["Nadir-Bild"], ascending=True)
fvc_df.to_excel( excel_file, index=False)

print("\n" + excel_file, "ausgegeben.")